# Simple example: Transformations from SQL to parametrized circuit

## Parsing

We parse the queries with ANTRL framework which is an extensive general-purpose parsing tool. We selected the SQLite grammar for its simplicity in ANTRL prewritten grammars. Because the core features of SQL are the same for any relational database, this code does not depend on system's SQL dialect.

So far, the module can deal with queries that have a SELECT-FROM-WHERE structure without substatements.

In [1]:
from antlr4 import *
from SQLiteLexer import SQLiteLexer
from SQLiteParser import SQLiteParser
from SQLiteParserListener import SQLiteParserListener
import json
import os
import glob
from pathlib import Path
from discopy import Ty, Box, Functor, Id, Swap, hypergraph, Cup
from functools import reduce
import numpy as np
import sympy
from discopy.quantum.pennylane import to_pennylane, PennyLaneCircuit
from sympy import default_sort_key
import pennylane as qml
import torch
from lambeq import IQPAnsatz
from pennylane.drawer import draw,draw_mpl,tape_mpl
from discopy.utils import dumps, loads

input_file = "simple_examples/sql5.sql"

input_stream = FileStream(input_file)
lexer = SQLiteLexer(input_stream)
stream = CommonTokenStream(lexer)
parser = SQLiteParser(stream)
tree = parser.parse()

print("Whole parse tree: ")
print(tree.toStringTree(recog=parser))

NameError: name 'Lexer' is not defined

## Diagrammatic representation of SQL language elements in context free grammar

We walk the tree and collect the elements in a monoidal category. This creates a diagrammatic representation for the parsed SQL query. `SQLiteParserListener` class implements DisCoPy construction. First we collect the abstract parse tree but we also modify it so that we collect SELECT and the columns into the same element, FROM and tables into the same element and WHERE and the filtering clauses into the same element. This interpretation follows the [railroad diagram representation](https://www.sqlite.org/syntaxdiagrams.html#select-core) more accurately than the current parsing. It also enables us to map the parse tree correctly later.

In [ ]:
this_folder = os.path.abspath(os.getcwd())
total_dim = 0
walker = ParseTreeWalker()
listener = SQLiteParserListener(parser)
walker.walk(listener, tree)
diagram = listener.get_final_diagram().dagger()
width = diagram.width()
height = diagram.depth()
dim = 4*max(width, height)
total_dim += dim
diagram.draw(figsize=(dim, dim), fontsize = 17, fontsize_types = 17) #path = this_folder + "\\figures\\bugging.png")

## Map context free grammar representations to pregroup representations

Because the abstract syntax tree contains lots of unnecessary information for our purposes, we will functorially rewrite it. This functorial rewriting process is just something that we have developed especially for this work and it is open to discussion if there exists a more suitable rewrite mapping. Also, the parts `select_main`, `from_main` and `where_main` are especially designed for this work although they are visible in the SQLite railroad diagrams without any special name.

Functor simply describes how the boxes and types are mapped. In this rewriting process we want to simplify the abstract syntax tree.

In [ ]:
#from cfg_alias_rewriting_mappings import alias_object_mapping, alias_morphism_mapping

#Rewriter = Functor(ob = lambda x: alias_object_mapping(x), ar = lambda f: alias_morphism_mapping(f))
#cfg_diagram = Rewriter(diagram)
#cfg_diagram.draw(figsize=(15, 15)) #, path = this_folder + "\\figures\\pregroup_figure.png")

#back_n_forth = lambda f: hypergraph.Diagram.upgrade(f).downgrade()
#back_n_forth(cfg_diagram).draw(figsize=(15, 15))

In [ ]:
from pregroupFunctorMappings import count_boxes, object_mapping, arrow_mapping

num_of_result_columns = count_boxes(diagram, "result-column")
num_of_result_columns += count_boxes(diagram, "result-column-with-alias")
num_of_tables = count_boxes(diagram, "table")
num_of_tables += count_boxes(diagram, "table-with-alias")

Rewriter = Functor(ob = lambda x: object_mapping(x, num_of_result_columns, num_of_tables), ar = lambda f: arrow_mapping(f, num_of_result_columns, num_of_tables))
pregroup_diagram = Rewriter(diagram)
width = pregroup_diagram.width()
height = pregroup_diagram.depth()
dim = 3*max(width, height)
total_dim += dim
pregroup_diagram.draw(figsize=(dim, dim), fontsize = 17, fontsize_types = 17)#, path = this_folder + "\\figures\\pregroup_figure.svg")

## Convert pregroup representations to circuits

Following the ideas of the paper A Quantum Natural Language Processing Approach to Musical Intelligence, we can reduce the number of qubits by rewriting the diagram and removing cups.

An assumption is that every box contains a connection to a cup. On the other hand, the SELECT-box does not need to be changed. Thus for every box (which is not a cup or the select box) we "raise the first leg on top of the box". This process creates snakes which the normalization process automatically removes. This rewriting process ensures that the cups are removed from the diagram and we can use less qubits.

In [ ]:
def cup_remove_arrow_mapping(box):
    if box.name.lower() == 'select':
        return box
    elif not box.cod:
        domain = box.dom
        raised_leg = Ty(domain[0])
        new_domain = reduce(lambda x, y : x @ Ty(y), domain[1:], Ty())
        new_box = Id(raised_leg) @ Box(box.name, new_domain, raised_leg.l)\
        >> Cup(raised_leg, raised_leg.l)
        return new_box
    return box

def cup_remove_arrow_mapping2(box):
    if box.cod == box.dom == Ty('n'):
        return Id(box.cod)
    return box

cup_removal_functor = Functor(ob = lambda x: x, ar = lambda f: cup_remove_arrow_mapping(f))
cup_removal_functor2 = Functor(ob = lambda x: x, ar = lambda f: cup_remove_arrow_mapping2(f))

In [ ]:
cupless_pregroup_diagram = cup_removal_functor(pregroup_diagram.normal_form()).normal_form()
cupless_pregroup_diagram = cup_removal_functor2(cupless_pregroup_diagram).normal_form()
width = cupless_pregroup_diagram.width()
height = cupless_pregroup_diagram.depth()
dim = 3*max(width, height)
total_dim += dim
cupless_pregroup_diagram.draw(figsize=(dim, dim), fontsize = 17, fontsize_types = 17, path = this_folder + "\\figures\\cup_removed_pregroup_figure.png")

cup_removed_pregroup_folder_name = "join-order-benchmark-diagrams//cup-removed-pregroup-diagrams"
with open(this_folder + "\\" + cup_removed_pregroup_folder_name + "\\" + "4b" + ".json", 'w') as outfile:
    json.dump(json.loads(dumps(cupless_pregroup_diagram)), outfile)

### Pregroup grammar to circuit ansatz functor

In [ ]:
n, s = Ty('n'), Ty('s')
ansatz = IQPAnsatz({n: 1, s: 1}, n_layers=1, n_single_qubit_params=1)
circuit_diagram = ansatz(cupless_pregroup_diagram)
width = circuit_diagram.width()
height = circuit_diagram.depth()
dim = 0.9*max(width, height)
total_dim += dim
circuit_diagram.draw(figsize=(dim, dim), fontsize = 17, fontsize_types = 17) #, path = this_folder + "\\figures\\circuit_figure2.png")

## Draw process in single figure

In [ ]:
from discopy.drawing import equation

equation_diagram = equation(diagram, 
                            cupless_pregroup_diagram, 
                            circuit_diagram, 
                            symbol = '→', 
                            fontsize = 12,
                            figsize=(23, 9),
                            fontsize_types = 12, 
                            path = this_folder + "\\figures\\equation.svg",
                            scale = (1.2, 0.7),
                            pad = (0,0),
                            draw_type_labels = False)

## Transforming circuit in Pennylane

In [ ]:
parameters = circuit_diagram.free_symbols
parameters = sorted(parameters, key=default_sort_key)

print("Number of parameters: ", len(parameters))

pennylane_circuit = to_pennylane(circuit_diagram)
qml_circuit = pennylane_circuit.make_circuit()
print(type(qml_circuit.func))
tot_qubits = qml_circuit.device.num_wires

In [ ]:
SEED = 0
rng = np.random.default_rng(SEED)
x0 = np.array(rng.random(len(parameters)))
np.random.seed(SEED)
x0 = [torch.FloatTensor([x]) for x in x0]

In [ ]:
@qml.qfunc_transform
def change_measurement_to_sample(tape):
    for op in tape.operations + tape.measurements + tape.observables:
        print(str(op))
        if str(op) == "state(wires=[])":
            qml.sample()
        else:
            qml.apply(op)

In [ ]:
dev = qml.device("default.qubit", wires=tot_qubits, shots=4)
params = pennylane_circuit.param_substitution(parameters, x0)

def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(y, wires=1)
    return qml.state()

new_circuit_func = change_measurement_to_sample(my_quantum_function)

target_system = qml.QNode(new_circuit_func, dev)
#print(target_system)
print(target_system(1,1))

#print(type(qml_circuit.func(params)))

specs_func = qml.specs(target_system)
print(json.dumps(specs_func(1,1), indent = 4))

#@qml.qnode(dev)
#def target_system(params):
#    transformed_qnode.func(params)
#    return qml.sample()


#transformed_qnode(params)

In [ ]:
dev_for_drawing = qml.device("qiskit.aer", wires=tot_qubits, backend='unitary_simulator')

#pennylane_circuit.draw(parameters, x0)
params = pennylane_circuit.param_substitution(parameters, x0)
print(params)
#fig, ax = qml.draw_mpl(qml_circuit)(params)

qml_circuit_for_drawing = qml.QNode(qml_circuit.func, dev_for_drawing)
print(qml_circuit.func)
qml_circuit_for_drawing(params)
dev_for_drawing._circuit.draw(output = "latex_source")

## Optimizing parameters

As the circuit ansatz produced by Lambeq shows, we need to perfom post-selection on the other qubits except the first ones.

In [ ]:
#dev = qml.device("default.qubit", wires=tot_qubits, shots=4)

#@qml.qnode(dev)
#def target_system(params):
#    qml_circuit.func(params)
#    return qml.sample()

#target_system(params)